This Notebook evaluates the study comparing the different variants of numerical fluxes for a Mach 4 supersonic flow over a blunt body presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024) 

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_FluxStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_FluxStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy'.


{ Session Count = 4; Grid Count = 4; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_FluxStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 21:36:48Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:23 PM	c2e02d32...
#1: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:09 PM	f5ed7ee5...
#2: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:34 PM	e575a6fe...
#3: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:48 PM	66a604d1...


In [5]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

c2e02d32-e0ea-4c5b-8347-6bee141b487e

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [8]:
var ses=sessions.Where(s => (s.SuccessfulTermination &&s.Name.Contains("bFlx0")&&s.Name.Contains("wFLx0"))); // the one plotted in the publication
ses


#0: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:23 PM	c2e02d32...
#1: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:09 PM	f5ed7ee5...
#2: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx2-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:34 PM	e575a6fe...
#3: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx3-wFLx0-bFlx0-Fphi0	4/3/2024 9:36:48 PM	66a604d1...


# A Helper Function to obtain different plot formats

In [9]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [10]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M511.2,57.1 L564.6,57.1 M45.6,185.2 L51.1,163.8 L56.6,168.3 L62.2,168.5 L67.7,168.6 L73.2,168.6
 L78.7,168.6 L84.3,168.6 L89.8,168.6 L95.3,168.6 L100.8,168.6 L106.3,168.6 L111.9,168.6 L117.4,168.6
 L122.9,169.0 L128.4,169.5 L134.0,176.1 L139.5,178.2 L145.0,179.3 L150.5,180.3 L156.1,180.4 L161.6,180.8
 L167.1,181.5 L172.6,180.7 L178.1,182.7 L183.7,183.2 L189.2,183.2 L194.7,183.2 L200.2,183.2 L205.8,183.4
 L211.3,183.3 L216.8,182.7 L222.3,182.8 L227.8,182.9 L233.4,182.9 L238.9,182.7 L244.4,182.8 L249.9,182.8
 L255.5,182.8 L261.0,182.8 L266.5,182.7 L272.0,182.7 L277.5,182.7 L283.1,182.6 L288.6,182.7 L294.1,182.6
 L299.6,182.6 L305.2,182.6 L310.7,182.6 L316.2,182.6 L321.7,182.5 L327.2,182.6 L332.8,182.6 L338.3,182.6
 L343.8,182.6 L349.3,182.6 L354.9,182.5 L360.4,182.5 L365.9,182.5 L371.4,182.6 L377.0,182.6 L382.5,182.6
 L388.0,182.6 L393.5,182.5 L399.0,182.5 L404.6,182.5 L410.1,182.5 L415.6,182.5 L421.1,182.5 L426.7,182.5
 L432.2,182.5 L437.7,182.4 L443.2,182.4 L448.7,182.4 L454.3,182.4 L459.8,182.4 L465.3,182.5 L470.8,182.5
 L476.4,182.5 L481.9,182.5 L487.4,182.5 L492.9,182.5 L498.4,182.5 L504.0,182.5 L509.5,182.5 L515.0,182.5
 L520.5,182.5 L526.1,182.5 L531.6,182.5 L537.1,182.5 L542.6,182.5 L548.1,182.5 L553.7,182.5 L559.2,182.5
 L564.7,182.5 L570.2,182.5 L575.8,182.5 L581.3,182.5 L586.8,182.5 L592.3,182.5 L597.9,182.5 L603.4,182.5
 L608.9,182.5 L614.4,182.5 L619.9,182.5 L625.5,182.5 L631.0,182.5 L636.5,182.5 L642.0,182.5 L647.6,182.5
 L653.1,182.5 L658.6,182.5 L664.1,182.5 L669.6,182.5 L675.2,182.5 L680.7,182.5 L686.2,182.5 L691.7,182.5
 L697.3,182.5 L702.8,182.5 L708.3,207.5 L713.8,213.1 L719.3,239.3 L724.9,283.7 L730.4,320.2 L735.9,357.3
 L741.4,396.7 L747.0,400.8 L752.5,399.7 L758.0,402.3 L763.5,402.2 L769.0,399.2 L774.6,401.6 L780.1,401.8
 L785.6,399.7 L791.1,402.0 L796.7,401.8 L802.2,399.9 L807.7,402.0 L813.2,401.9 L818.8,399.9 L824.3,402.1
 L829.8,401.9 L835.3,399.9 L840.8,402.0 L846.4,401.9 L851.9,399.9 L857.4,402.1 L862.9,401.9 L868.5,400.0
 L874.0,487.0 L879.5,500.5 L885.0,523.4 L890.5,527.5 L896.1,528.2 L901.6,528.1 L907.1,528.1 L912.6,528.0
 L918.2,528.1 L923.7,528.1 L929.2,528.1 L934.7,528.1 L940.2,528.1 L945.8,528.1 L951.3,528.1 L956.8,528.1
 L962.3,528.1 L967.9,528.1 L973.4,528.1 L978.9,528.1 L984.4,528.1 L989.9,528.1 L995.5,528.1 L1001.0,528.1
 L1006.5,528.1 L1012.0,528.1 L1017.6,528.1 L1023.1,528.1 L1028.6,528.1 L1034.1,528.1 L1039.7,528.1 L1045.2,528.1
 L1050.7,528.1 L1056.2,528.1 L1061.7,528.1 L1067.3,528.1 L1072.8,528.1 L1078.3,528.1 L1083.8,528.1 L1089.4,528.1
 L1094.9,528.1 L1100.4,528.1 L1105.9,528.1 L1111.4,528.1 L1117.0,528.1 L1122.5,528.1 L1128.0,528.1 L1133.5,528.1
 L1139.1,528.1 L1144.6,528.1 L1150.1,528.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M511.2,81.1 L564.6,81.1 M45.6,185.2 L51.1,164.0 L56.6,169.1 L6

## Residual History

In [11]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    lts.EnResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M22.4,57.1 L75.8,57.1 M53.9,79.6 L57.4,82.8 L60.9,84.4 L64.3,84.5 L67.8,84.5 L71.3,84.5
 L74.8,84.5 L78.3,84.5 L81.7,84.5 L85.2,84.5 L88.7,84.5 L92.2,84.5 L95.7,84.5 L99.2,84.5
 L102.6,84.6 L106.1,84.6 L109.6,91.5 L113.1,92.2 L116.6,95.5 L120.0,97.3 L123.5,97.5 L127.0,98.3
 L130.5,99.9 L134.0,109.4 L137.4,111.2 L140.9,115.7 L144.4,111.7 L147.9,117.4 L151.4,117.4 L154.8,119.3
 L158.3,122.1 L161.8,126.5 L165.3,124.8 L168.8,127.1 L172.3,128.2 L175.7,129.6 L179.2,130.0 L182.7,130.7
 L186.2,131.1 L189.7,132.7 L193.1,136.8 L196.6,137.3 L200.1,137.7 L203.6,138.8 L207.1,139.0 L210.5,140.9
 L214.0,141.0 L217.5,141.3 L221.0,142.1 L224.5,142.7 L228.0,143.4 L231.4,143.6 L234.9,143.7 L238.4,143.8
 L241.9,144.1 L245.4,144.4 L248.8,142.6 L252.3,142.9 L255.8,143.2 L259.3,143.5 L262.8,143.8 L266.2,144.2
 L269.7,145.2 L273.2,147.1 L276.7,147.2 L280.2,159.8 L283.6,159.8 L287.1,159.8 L290.6,159.9 L294.1,159.9
 L297.6,159.9 L301.1,160.1 L304.5,173.6 L308.0,173.6 L311.5,173.8 L315.0,174.1 L318.5,175.2 L321.9,174.8
 L325.4,174.8 L328.9,174.8 L332.4,174.7 L335.9,175.9 L339.3,175.9 L342.8,175.9 L346.3,175.9 L349.8,175.4
 L353.3,175.5 L356.7,175.6 L360.2,176.2 L363.7,176.1 L367.2,176.2 L370.7,176.3 L374.2,176.2 L377.6,176.3
 L381.1,176.5 L384.6,195.0 L388.1,195.0 L391.6,195.0 L395.0,195.0 L398.5,195.0 L402.0,195.0 L405.5,195.0
 L409.0,195.1 L412.4,195.2 L415.9,195.8 L419.4,196.5 L422.9,196.6 L426.4,196.7 L429.8,197.2 L433.3,197.6
 L436.8,197.6 L440.3,197.8 L443.8,209.2 L447.3,209.2 L450.7,209.2 L454.2,209.2 L457.7,209.2 L461.2,209.2
 L464.7,209.2 L468.1,209.2 L471.6,82.6 L475.1,86.6 L478.6,89.3 L482.1,93.3 L485.5,96.7 L489.0,108.7
 L492.5,125.7 L496.0,129.6 L499.5,137.3 L502.9,144.8 L506.4,130.0 L509.9,146.7 L513.4,147.9 L516.9,129.0
 L520.4,150.9 L523.8,148.6 L527.3,129.3 L530.8,150.9 L534.3,148.3 L537.8,129.3 L541.2,151.3 L544.7,148.4
 L548.2,129.4 L551.7,151.1 L555.2,148.3 L558.6,129.4 L562.1,151.3 L565.6,148.4 L569.1,129.4 L572.6,151.2
 L576.1,115.1 L579.5,96.1 L583.0,133.9 L586.5,165.9 L590.0,175.3 L593.5,211.6 L596.9,243.5 L600.4,303.5
 L603.9,302.7 L607.4,303.6 L610.9,304.3 L614.3,319.1 L617.8,321.5 L621.3,322.5 L624.8,330.8 L628.3,332.4
 L631.7,335.0 L635.2,347.9 L638.7,399.1 L642.2,399.3 L645.7,400.9 L649.2,405.6 L652.6,406.6 L656.1,406.7
 L659.6,411.5 L663.1,411.7 L666.6,413.8 L670.0,413.9 L673.5,413.4 L677.0,415.2 L680.5,415.2 L684.0,416.1
 L687.4,416.2 L690.9,417.4 L694.4,417.1 L697.9,417.0 L701.4,416.9 L704.8,417.2 L708.3,416.6 L711.8,417.6
 L715.3,418.5 L718.8,418.0 L722.3,418.1 L725.7,417.9 L729.2,418.2 L732.7,418.1 L736.2,418.8 L739.7,419.0
 L743.1,418.6 L746.6,419.2 L750.1,418.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0

### Plot ShadowFields

In [12]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm;
